In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import mlflow
import dagshub
from dagshub import dagshub_logger
import json
import os
import keras
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from kerastuner.tuners import BayesianOptimization

2025-09-28 11:21:30.655750: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-28 11:21:31.266054: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-28 11:21:33.616043: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/tmp/ipykernel_27916/1167736315.py:15: DeprecationWarning: `import kerastuner` is deprecate

In [4]:
# Dagshub and MLflow setup
mlflow.set_tracking_uri("https://dagshub.com/alfoCaiazza/churn_prediction.mlflow")
dagshub.init(repo_owner="alfoCaiazza", repo_name="churn_prediction", mlflow=True)
dagshub_logger = dagshub_logger(metrics_path="metrics", hparams_path="params")

Initialized MLflow to track repo "alfoCaiazza/churn_prediction"

Repository alfoCaiazza/churn_prediction initialized!